In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
  print("_THIS_",type(features))
  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits/temprature, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"]),
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




In [2]:
#cross entropy between two logits with temperature applied
#returns a 1d array with loss for each row in the inputs
def cross_entropy2(tar_soft_t, pred_logits):
   pred_soft_t = tf.nn.softmax(pred_logits/temprature)
#    tar_soft = tf.nn.softmax(tar_logits_t/temprature)
   pred_log = -1 * tf.log(pred_soft_t)
   product =tf.multiply(tar_soft_t,pred_log)
#    print("Swapnil", pred_log,tar_soft_t,product)
   return tf.reduce_mean(product)
    
def custom_loss(y_true, pred_logits, tar_soft_t):
  return  loss_weight * tf.losses.sparse_softmax_cross_entropy(labels = y_true, logits = pred_logits)+(1-loss_weight)*cross_entropy2(tar_soft_t,pred_logits)


In [5]:
def getFilterData(f,l):
  sess=tf.InteractiveSession()
  tf.train.start_queue_runners(sess)
  data_s=f['x'].eval()
  out_s=l.eval()
  sess.close()
  return data_s,out_s


def student_model_fn(features, labels, mode):

  print(features,labels)
  data_swap,out_swap=getFilterData(features,labels)
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  if mode == tf.estimator.ModeKeys.TRAIN:
      eval_teacher_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x":data_swap},
          y=out_swap,
          batch_size=100,
          shuffle=False)

      eval_teacher=mnist_classifier.evaluate(input_fn=eval_teacher_fn)
      outlog.write('%f;' % eval_teacher['accuracy'])
      outlog.write('%f;' % eval_teacher['loss'])
      outlog.write('\n')
      predictions=mnist_classifier.predict(input_fn=eval_teacher_fn)
      teacher_pred=list(predictions)
      teacher_soft=[ p['probabilities'] for p in teacher_pred]
      temp=np.array(teacher_soft)
      
      teacher_soft_t=tf.convert_to_tensor(temp)


  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 8])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=256, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  #   loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  
    
#   student_pred=predictions['probabilities']
#   sess1=tf.InteractiveSession()
#   tf.train.start_queue_runners(sess1)
#   student_soft=student_pred.eval()
#   sess1.close()  
  if mode == tf.estimator.ModeKeys.TRAIN:
#     pred_soft_t=tf.nn.softmax(logits, name="softmax_tensor")
#     print('a',labels,'b', logits,'c', teacher_soft_t ,'d', pred_logits_t)
    loss=custom_loss(labels, logits,teacher_soft_t)
  else:
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  
    
  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"]),
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




In [6]:
cifar_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="cifar_convnet_model_test1")

mnist = tf.keras.datasets.cifar10.load_data()
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
temp_list=[1,2,5,10]
loss_weight_list=[0,0.25,0.5,0.75,1]
p=-1
l=-1
def getdatap():
    global p
    p += 1
    if p>len(train_labels)/100:
        p=0
    data=train_data[p*100:(p+1)*100]
    labels=train_labels[p*100:(p+1)*100]
    
    datatensor=tf.convert_to_tensor(data)
    labeltensor=tf.convert_to_tensor(labels)
    
    
    return {'x':datatensor},labeltensor

out = open('swapout4.csv', 'w')
outlog = open('out_log.csv', 'w')

for i in temp_list:
    for j in loss_weight_list:
        temprature=i
        loss_weight=j
        # Create the Estimator
        student_classifier = tf.estimator.Estimator(
          model_fn=student_model_fn, model_dir="./mnist_convnet_student_mod_t_"+str(i)+"_w_"+str(j))

        
#         eval_train_fn = tf.estimator.inputs.numpy_input_fn(
#           x={"x": train_data[:100]},
#           y=train_labels[:100],
#           num_epochs=1,
#           batch_size=100,
#           shuffle=False)
#         train_result=mnist_classifier.evaluate(input_fn=eval_train_fn)
#         #print("\n \n \n Train 100",train_result)
#         print("\n\n\nLABEL FUNCTION",train_labels[:100])
#         print("\n\n\nTRAIN DATA FUNCTION",train_data[0])
        
        # Train the model
#         train_input_fn = tf.estimator.inputs.numpy_input_fn(
#           x={"x": train_data},
#           y=train_labels,
#           batch_size=100,
#           num_epochs=None,
#           shuffle=True)
#         print(train_data,train_labels,eval_data,eval_labels)

        
        student_classifier.train(
          input_fn=getdatap,
          steps=5000,
          hooks=None)
        # print("eva",type(eval_data))

        # Evaluate the model and print results
        eval_train_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": train_data},
          y=train_labels,
          shuffle=False)
        train_result=student_classifier.evaluate(input_fn=eval_train_fn)
        
        out.write('%f;' % i)
        out.write('%f;' % j)
        out.write('%f;' % train_result['accuracy'])
        out.write('%f;' % train_result['loss'])
        
        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": eval_data},
          y=eval_labels,
          num_epochs=1,
          shuffle=False)
        eval_result=student_classifier.evaluate(input_fn=eval_input_fn)
        # output=list(predictions)
        
        out.write('%f;' % eval_result['accuracy'])
        out.write('%f;' % eval_result['loss'])
#         acc=eval_result
#         out.write(eval_result)
        out.write('\n')
        
out.close()
outlog.close()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'cifar_convnet_model_test1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020C13351AC8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
170500096/170498071 [==============================]170500096/170498071 [==============================] - 319s 2us/step



AttributeError: 'tuple' object has no attribute 'train'

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": eval_data},
  y=eval_labels,
  shuffle=False)
eval_result=student_classifier.evaluate(input_fn=eval_input_fn)
 

In [ ]:
 
def custom_loss(y_true, pred_logits, tar_soft_t, pred_logits_t):
#    print(tf.losses.sparse_softmax_cross_entropy(labels = y_true, logits = pred_logits))
#    print(l * cross_entropy2(tar_soft_t, pred_logits_t))
   return tf.losses.sparse_softmax_cross_entropy(labels = y_true, logits = pred_logits) + lam * cross_entropy2(tar_soft_t, pred_logits_t)
mnist_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model_test1")

mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
lam_list=[0,1,2,4,6,8,9,10]
temprature=2
out = open('out_lam.csv', 'w')
for i in lam_list:
    lam=i
    # Create the Estimator
    student_classifier = tf.estimator.Estimator(
      model_fn=student_model_fn, model_dir="./mnist_convnet_student_t_"+str(2)+"_lam_"+str(i))

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
    print(train_data,train_labels,eval_data,eval_labels)

    student_classifier.train(
      input_fn=train_input_fn,
      steps=20000,
      hooks=None)
    # print("eva",type(eval_data))

    # Evaluate the model and print results
    eval_train_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      num_epochs=1,
      shuffle=False)
    train_result=student_classifier.evaluate(input_fn=eval_train_fn)

    out.write('%f;' % i)
    out.write('%f;' % j)
    out.write('%f;' % train_result['accuracy'])
    out.write('%f;' % train_result['loss'])

    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
    eval_result=student_classifier.evaluate(input_fn=eval_input_fn)
    # output=list(predictions)

    out.write('%f;' % eval_result['accuracy'])
    out.write('%f;' % eval_result['loss'])
#         acc=eval_result
#         out.write(eval_result)
    out.write('\n')

out.close()
        

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": train_data},
  y=train_labels,
  batch_size=100,
  num_epochs=10,
  shuffle=False)
eval_result=mnist_classifier.evaluate(input_fn=eval_input_fn)
 